In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

import pandas as pd
import numpy as np
import sys
%pip install pandasql
from pandasql import sqldf

sys.path.extend(['../src/preprocess', '../config'])

region = boto3.Session().region_name
session = sagemaker.Session()
sm = boto3.Session().client(service_name='sagemaker',region_name=region)

import helpers.instance as ins
import helpers.s3 as s3_helper
import helpers.utils as ut
import helpers.athena as at

# READ YAML FILES AND STORE RELEVANT PATHS in dictionaries
dataset_cfg = ins.read_config('../config/datasets.yaml') 
config_cfg = ins.read_config('../config/config.yaml')
sql_cfg = ins.read_config('../config/sql.yaml')

Note: you may need to restart the kernel to use updated packages.


In [2]:
join_on1 = ['unit_num', 'partition_key']
join_on2 = ['unit_num']
joining = 'left'

In [3]:
region = boto3.Session().region_name
region

'us-east-1'

In [4]:
input_files = [dataset_cfg['raw']['repair_base']['input_file'], dataset_cfg['raw']['base_query']['input_file'], dataset_cfg['raw']['engine_hours']['input_file'], dataset_cfg['raw']['postsaleissue']['input_file']]
output_paths = [dataset_cfg['raw']['repair_base']['output_paths'], dataset_cfg['raw']['base_query']['output_paths'], dataset_cfg['raw']['engine_hours']['output_paths'], dataset_cfg['raw']['postsaleissue']['output_paths']]

In [5]:
# passing Athena variables
database=['ptledw_playarea', 'ptl_maintenance', 'ptl_connfleet', 'proactive_maint_db', 'ptl_adhoc']
workgroup='WG-Advanalytics'
tablename=['d_location_master','constellation_rules','ufch_depup', 'archive_pd_inference_scenario']
#query_string = 'select * from ptl_maintenance.constellation_rules'
#query_string = 'select * from {}.{}'.format(database[0],tablename[0])
# 'select * from ptledw_playarea.d_location_master'

In [6]:
prepped_base_query = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_query_prepped/')

prepped_repair_base = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/repair_base_cleaned/')

join_on1 = ['unit_num', 'partition_key']
join_on2 = ['unit_num']
joining = 'left'

In [7]:
based_prepped_joined = ut.join(prepped_df=prepped_base_query, prepped_df2=prepped_repair_base, keycolumns=join_on1, jointype=joining)
based_prepped_joined
#base_repair_joined_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/based_prepped_joined/'
#s3_helper.persist_file_to_path(df=base_repair_joined, path=base_repair_joined_path, filetype='parquet')

AttributeError: module 'helpers.utils' has no attribute 'join'

In [ ]:
replacing = ['labor_hours', 'parts_cost', 'labor_cost', 'outside_cost', 'major_pm', 'big_repair', 'accidentsinci', 'total_repairs']

based_prepped_joined_prepared = ut.removing_blanks(df=based_prepped_joined, column_list=replacing)
based_prepped_joined_prepared

#new_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall/'
#s3_helper.persist_file_to_path(df=base_repair_roadcall, path=new_path, filetype='parquet')

In [ ]:
s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/repair_base_cleaned/')

In [ ]:
prepped_roadcalls = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/roadcalls_prepped/')

base_repair_roadcall = ut.join(prepped_df=based_prepped_joined_prepared, prepped_df2=prepped_roadcalls, keycolumns=join_on1, jointype=joining)

#new_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall/'
#s3_helper.persist_file_to_path(df=base_repair_roadcall, path=new_path, filetype='parquet')

In [ ]:


#base_repair_roadcall_oil = left_join(prepped_df=base_repair_roadcall, prepped_df2=prepped_job_oil, keycolumns= join_on2)

#repair_roadoil_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil/'
#s3_helper.persist_file_to_path(df=base_repair_roadcall_oil, path=repair_roadoil_path, filetype='parquet')

In [ ]:
base_repair_roadcall_oil = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil/')

In [ ]:
replacing_cont = ['road_call_cnt', 'downtime', 'handle_time', 'customer_rebilled_cnt', 'po_amt']
base_repair_roadcall_oil_prep = ut.removing_blanks(base_repair_roadcall_oil, replacing_cont)
base_repair_roadcall_oil_prep

#base_repair_roadcall_oil_prep_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil_prep/'
#s3_helper.persist_file_to_path(df=base_repair_roadcall_oil_prep, path=base_repair_roadcall_oil_prep_path, filetype='parquet')

In [ ]:
#assuming ...oil_prep is properly cleaned

In [ ]:
prepped_pm_combined = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/pm_combined_prepped/').sample(n = 100000)

reading_base_repair_roadcall_oil_prep = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil_prep/').sample(n = 100000)

In [ ]:
base_repair_roadcall_oil_prep_joined = ut.join(prepped_df=reading_base_repair_roadcall_oil_prep, prepped_df2=prepped_pm_combined, jointype=joining, keycolumns= join_on2)
base_repair_roadcall_oil_prep_joined

In [ ]:
#base_repair_roadcall_oil_prep_joined_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil_prep_joined/'
#s3_helper.persist_file_to_path(df=base_repair_roadcall_oil_prep_joined, path=base_repair_roadcall_oil_prep_joined_path, filetype='parquet')

In [ ]:
prepped_bill_date = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/bill_date_prepped/')

base_repair_roadcall_oil_pm_bill = ut.join(prepped_df=base_repair_roadcall_oil_prep_joined, prepped_df2=prepped_bill_date, keycolumns= join_on2, jointype=joining)
base_repair_roadcall_oil_pm_bill
#base_repair_roadcall_oil_pm_bill_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil_pm_bill/'
#s3_helper.persist_file_to_path(df=base_repair_roadcall_oil_pm_bill, path=base_repair_roadcall_oil_pm_bill_path, filetype='parquet')

In [ ]:
def editing_final(file):
    '''
        
    '''
    df = file
    mode_val_fill = ['unit_size', 'unit_fuel_type']

    df['unit_sold_pk'] = df['unit_sold_date'].dt.year *100 + df['unit_sold_date'].dt.month
    df[df['partition_key']<=df['unit_sold_pk']]
    df['billed_pk'].fillna(0, inplace=True)
    df.sort_values(by=['unit_num', 'partition_key'], ascending=(False, False), inplace=True)

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    '''
    - Convert partition_key to date
    - Get timelapse between last record in EOM and current record, for grouping purpose in final query
    '''
    df['partition_date'] = pd.to_datetime(df['partition_key'], format='%Y%m')
    df['last_date'] = df.groupby('unit_num')['partition_date'].transform('first')
    df['time_lapse_month'] = np.round((df['last_date'] - df['partition_date'])/np.timedelta64(1,'M'))

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    '''
    Dropping unnecesary columns and filling categorical null values in columns with mode of columns
    '''
    df.drop(columns=['last_billed_date', 'last_date', 'partition_date'], inplace=True)

    for col in mode_val_fill:
        df[col].fillna(df[col].mode()[0],inplace=True)

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    combined_df = df # For this sample code, simply copy input to output
    return combined_df

#read_base_repair_roadcall_oil_pm_bill = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil_pm_bill/')
prefinal_data = editing_final(file=base_repair_roadcall_oil_pm_bill)
prefinal_data

#prefinal_data_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/prefinal_data/'
#s3_helper.persist_file_to_path(df=prefinal_data, path=prefinal_data_path, filetype='parquet')

In [ ]:
def finalized_data(file):
    '''
        
    '''
    
    df = file
    df['total_cost'] = df['parts_cost'] + df['labor_cost'] + df['outside_cost']

    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    merged = sqldf('''select 
        unit_num, 
        sum(case when time_lapse_month between 1 and 24 then total_cost else 0 end) as Tot_cost_24,
        sum(case when time_lapse_month between 1 and 12 then total_cost else 0 end) as Tot_cost_12,
        sum(case when time_lapse_month between 1 and 6 then total_cost else 0 end) as Tot_cost_6,
        sum(case when time_lapse_month between 1 and 3 then total_cost else 0 end) as Tot_cost_3,
        avg(case when time_lapse_month between 1 and 24 then total_cost else 0 end) as AvgT_cost_24,
        avg(case when time_lapse_month between 1 and 12 then total_cost else 0 end) as AvgT_cost_12,
        avg(case when time_lapse_month between 1 and 6 then total_cost else 0 end) as AvgT_cost_6,
        avg(case when time_lapse_month between 1 and 3 then total_cost else 0 end) as AvgT_cost_3,
        sum(case when time_lapse_month between 1 and 12 then outside_cost else 0 end) as Outside_cost_12,
        sum(case when time_lapse_month between 1 and 12 then handle_time else 0 end ) as total_time_call_12,
        sum(case when time_lapse_month between 1 and 12 then road_call_cnt else 0 end ) as total_call_12,
        sum(case when time_lapse_month between 1 and 12 then po_amt else 0 end ) as total_po_sum_12,
        sum(case when time_lapse_month between 1 and 12 then customer_rebilled_cnt else 0 end ) as time_customer_billed_12,

        sum(total_cost) as LTD_Total_cost,
        sum(labor_cost) as LTD_Labour_cost,
        sum(outside_cost) as LTD_Outside_cost,
        sum(parts_cost) as LTD_Parts_cost,
        sum(big_repair) as LTD_Big_repair,
        sum(major_pm) as LTD_Major_PM,
        sum(road_call_cnt) as LTD_Road_calls,
        sum(handle_time) as LTD_Handle_time,
        sum(total_repairs) as LTD_TOTAL_REPAIRS,
        sum(customer_rebilled_cnt) as LTD_customer_rebilled,
        sum(po_amt) as LTD_PO_Amount,
        sum(LABOR_HOURS) as LTD_Labour_hours,
        sum(accidentsinci) as LTD_Accients_incidents,
        avg(pm_early_ontime_percent) as avg_pm_ontime_percent,
        avg(critical_oil_sample_percentage) as avg_critical_oil_sample_percent,

        (sum(case when time_lapse_month between 0 and 6 then total_cost else 0 end) - sum(case when time_lapse_month between 7 and 12 then total_cost else 0 end) ) diff_prev6,
        (sum(case when time_lapse_month between 0 and 12 then total_cost else 0 end) - sum(case when time_lapse_month between 13 and 24 then total_cost else 0 end) ) diff_prev12,
        (sum(case when time_lapse_month between 0 and 3 then total_cost else 0 end) - sum(case when time_lapse_month between 4 and 6 then total_cost else 0 end) ) diff_prev3,
        avg(case when time_lapse_month == 0 then miles end) - avg(case when time_lapse_month == 6 then miles end) as milesdiff_1_6,
        (avg(case when time_lapse_month == 0 then total_cost end) - avg(case when time_lapse_month == 6 then total_cost end))/(avg(case when time_lapse_month == 0 then MILES end) - avg(case when time_lapse_month == 6 then MILES end)) as costPermile_1diff6,
        -- avg(case when time_lapse_month == 0 then ENGINE_HRS end) - avg(case when final_rank == 6 then ENGINE_HRS end) as Enginehrs_diff_1_6,
        (sum(case when time_lapse_month between 0 and 6 then total_repairs else 0 end) - sum(case when time_lapse_month between 7 and 12 then total_repairs else 0 end) ) Repairs_diff_prev6,
        -- avg( case when time_lapse_month == 0 then miles end)/avg(case when time_lapse_month == 0 then ENGINE_HRS end) as  cycle_time,

        -- (avg( case when time_lapse_month == 0 then miles end)/avg(case when time_lapse_month == 0 then engine_hrs end) - avg( case when time_lapse_month == 7 then miles end)/avg(case when time_lapse_month == 7 then engine_hrs end) ) as duty_cycle_change6months,
        avg(case when time_lapse_month == 0 then miles end) as Curr_miles,
        -- avg(case when time_lapse_month == 0 then engine_hrs end ) as curr_engine_hrs,
        avg(case when time_lapse_month == 0 then vehicle_age_total_months end ) as cur_vintage,

        sum(case when time_lapse_month between 0 and 24 then major_pm end) as MajorPM_24_sum,
        count(distinct location) as dloc_nt, 
        count(distinct region ) as dregion_cnt, 
        count(distinct area) as darea_cnt ,

        max(case when time_lapse_month = 0 then location end ) as LOCATION,
        max(case when time_lapse_month = 0 then area end ) as AREA,
        max(case when time_lapse_month = 0 then region end) as REGION,
        max(case when time_lapse_month = 0 then district end) as DISTRICT,
        max(case when time_lapse_month = 0 then unit_category end) as UNIT_CATEGORY,
        max(case when time_lapse_month = 0 then unit_size end) as UNIT_SIZE,
        max(case when time_lapse_month = 0 then product_line end) as PRODUCT_LINE,
        max(case when time_lapse_month = 0 then unit_make_code end) as UNIT_MAKE_CODE,
        max(case when time_lapse_month = 0 then unit_model_year end) as UNIT_MODEL_YEAR,
        max(case when time_lapse_month = 0 then unit_fuel_type end ) as UNIT_FUEL_TYPE,
        max(case when time_lapse_month = 0 then unit_body_type end ) as UNIT_BODY_TYPE,
        max(unit_sold_date) as UNIT_SOLD_DATE
    from 
        df
    group by unit_num
    ''')
    finalized_data_df = merged
    
    return finalized_data_df

#read_prefinal_data = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/prefinal_data/')
final_data = finalized_data(file=prefinal_data)
final_data

#final_data_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/final_data/'
#s3_helper.persist_file_to_path(df=final_data, path=final_data_path, filetype='parquet')

In [ ]:
post_sale_prepped = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/postsaleissue_cleaned/')

#read_final_data = s3_helper.read_parquet_from_path('s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/final_data/')

with_target = ut.join(prepped_df=final_data, prepped_df2=post_sale_prepped, jointype=joining, keycolumns= join_on2)
#with_target_path = 's3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Processed/base_repair_roadcall_oil/'
#s3_helper.persist_file_to_path(df=with_target, path=with_target_path, filetype='parquet')

In [ ]:
filtering_with_target = with_target.query("UNIT_CATEGORY in ('TRUCK', 'TRACTOR', 'TRAILER') ")

In [ ]:
replacing_final = ['flag_cameback']
filterdataset = ut.removing_blanks(filtering_with_target, replacing_final)
filterdataset['UNIT_SOLD_DATE'] = pd.to_datetime(filterdataset['UNIT_SOLD_DATE'], format='%Y-%m-%d %H:%M:%S.%f').sort_index()

filterdataset['UNIT_SOLD_DATE']

In [ ]:
#filterdataset.Start = pd.to_datetime()
target = pd.date_range('2019-01-01', '2022-01-01')
filterdataset['unit_sold_flag'] = np.where(filterdataset['UNIT_SOLD_DATE'].isin(target), 1, 0)
filteringdataset = filterdataset[filterdataset['unit_sold_flag'] == 1]
filterdf = filteringdataset.drop(columns=['unit_sold_flag'], inplace=True)

#target2 = pd.date_range('2021-09-01', '2021-12-31')
#filterdf['outvalidation_flag'] = np.where(filterdataset['UNIT_SOLD_DATE'].isin(target2), 1, 0 )
#filterdf2 = filterdf[filterdf['outvalidation_flag'] == 1]
#filterdf2 = filterdf.drop(columns=[unit])

start = '2021-09-01'
end = '2021-12-31'
test = (filterdf['UNIT_SOLD_DATE'] > start) | (filterdf['UNIT_SOLD_DATE'] <= end)

df = filterdf.loc[test]

In [ ]:
filteringdataset[filteringdataset['outvalidation_flag'] == 1 ]